In [ ]:
from transformers import CLIPVisionModel, AutoModel
model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
model = AutoModel.from_pretrained('facebook/dinov2-base')

In [ ]:
import torch
img = torch.rand(1, 3, 224, 224)

In [ ]:
output = model(img)

In [ ]:
output.last_hidden_state.shape

In [ ]:
output.pooler_output.shape

In [ ]:
from fish_benchmark.models import MediaClassifier

In [ ]:
model = MediaClassifier(5, 'dino', 'linear')

In [ ]:
model(img)

In [ ]:
from transformers import VideoMAEModel
import torch
video = torch.rand(1, 16, 3, 224, 224)
model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")
output = model(video)

In [ ]:
type(output)

In [ ]:
output.last_hidden_state.shape

In [ ]:
pooling = output.last_hidden_state.mean(dim=1)
pooling.shape

In [ ]:
from torch.nn import MultiheadAttention
import torch.nn as nn
def key_proj(x):
    return nn.Linear(x.shape[-1], 64)(x)
def value_proj(x):
    return nn.Linear(x.shape[-1], 128)(x)

token_embeddings = torch.rand(1, 1568, 768)
key = key_proj(token_embeddings)     # shape: [1, 1568, 64]
value = value_proj(token_embeddings) # shape: [1, 1568, 128]
attention = MultiheadAttention(768, 8, kdim=64, vdim = 128, batch_first=True)
query = torch.rand(1, 1, 768)
attn_output, attn_weights = attention(query, key, value)

In [ ]:
attn_output.shape

In [ ]:
model.config.hidden_size

In [ ]:
from fish_benchmark.models import AttentionBlock
import torch

video = torch.rand(1, 16, 3, 224, 224)
model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")
output = model(video)
output.last_hidden_state.shape

In [ ]:
output.last_hidden_state.shape

In [ ]:
head = AttentionBlock(embed_dim = model.config.hidden_size, output_dim = 101)
head(output.last_hidden_state).shape

In [23]:
from fish_benchmark.data.dataset import get_dataset
from fish_benchmark.models import get_input_transform
import yaml
DATASET_NAME = 'HeinFishBehavior'

config = yaml.safe_load(open('../config/datasets.yml', 'r'))
transform = get_input_transform('clip')
dataset = get_dataset(DATASET_NAME, config[DATASET_NAME]['path'], augs=None)



In [18]:
from transformers import AutoModelForImageClassification, AutoImageProcessor
import torch
model = AutoModelForImageClassification.from_pretrained("OpenGVLab/pvt_v2_b0")
image_processor = AutoImageProcessor.from_pretrained("OpenGVLab/pvt_v2_b0")
processed = image_processor(img, do_resize = False)
input = torch.tensor(processed["pixel_values"])
print(input.shape)
outputs = model(input)

torch.Size([1, 3, 1520, 2704])


In [19]:
type(outputs)

transformers.modeling_outputs.ImageClassifierOutput

In [20]:
outputs.logits.shape

torch.Size([1, 1000])

In [22]:
from transformers import AutoImageProcessor, Swinv2Model
import torch

image_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")
model = Swinv2Model.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")


inputs = image_processor(img, return_tensors="pt", do_resize=False)
print(inputs['pixel_values'].shape)

with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

torch.Size([1, 3, 1520, 2704])


[1, 4080, 768]

In [1]:
from fish_benchmark.data.dataset import get_dataset
from fish_benchmark.models import get_input_transform
import yaml
config = yaml.safe_load(open("../config/datasets.yml"))
DATASET = "MikeFramesPatched"
dataset = get_dataset(DATASET, path = config[DATASET]['path'], train=True)

/home/jth264/.conda/envs/benchmark/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from fish_benchmark.models import get_input_transform, get_pretrained_model
input_transform = get_input_transform('dino')
pretrained_model = get_pretrained_model('multipatch_dino')

In [4]:
frame, label = next(iter(dataset))

In [5]:
frame.shape

torch.Size([4, 760, 1352, 3])

In [6]:
input_transform(frame).shape

torch.Size([4, 3, 224, 224])

In [7]:
batch = input_transform(frame).unsqueeze(0)
print(batch.shape)

torch.Size([1, 4, 3, 224, 224])


In [8]:
pretrained_model(batch).shape

torch.Size([1, 1028, 768])

In [8]:
output = pretrained_model(input_transform(frame))

In [9]:
output.last_hidden_state.shape

torch.Size([4, 257, 768])

In [ ]:
#Normal dino
#[batch_size, channels, height, width] --> [batch_size, seq_len, latent_dim]
#Multiframe dino
#[batch_size, num_patches, channels, height, width] --> [batch_size, num_patches, seq_len, latent_dim]